In [22]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize
import os
import glob


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

In [2]:
df = pd.read_csv('../data/data_201_cols.csv').drop(['Unnamed: 0'], axis = 1)
df

,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,KDA100_2,KDA100_3,KDA100_4,KDA100_5,KDA200_6,KDA200_7,KDA200_8,KDA200_9,KDA200_10,winner
0,0,2741,3045,3545,2789,2026,4380,4114,3372,3419,...,2.42000,2.7000,2.3775,2.710000,2.162500,2.945000,2.885000,2.33250,2.450000,200
1,1,2888,4549,3947,3600,2678,3715,2891,3003,3084,...,2.29250,2.4425,2.6425,4.212500,1.952500,2.809167,2.905000,2.66750,3.180000,200
2,10,2664,3850,3712,4927,2841,4655,3186,4058,2968,...,2.44750,2.8375,2.6425,2.625000,2.667500,2.617500,3.015000,1.72500,2.710000,100
3,100,2814,4072,3289,5535,3880,3516,4619,2940,2653,...,2.61750,2.4000,3.1775,2.872500,2.055000,2.802500,2.230000,1.99000,2.744000,100
4,1000,3164,4548,3811,3998,3808,4078,3602,3068,2830,...,2.77000,2.4000,2.4000,2.850000,1.735000,2.945000,1.818750,2.64250,3.307500,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,995,3368,3486,3951,3682,2280,3232,3053,3835,3174,...,3.43500,2.4200,2.6925,2.872500,1.818750,2.593750,2.885000,2.37750,3.498750,100
7996,996,3842,3779,3663,4833,2739,3209,3274,3461,3667,...,2.59375,2.5925,2.5125,2.704286,1.742500,2.777143,1.820833,2.66750,3.131667,100
7997,997,3583,3761,3037,3093,2325,4141,3821,3343,4427,...,2.61750,2.8375,2.5125,2.675000,1.897500,2.527500,1.742500,2.45800,2.785000,200
7998,998,3000,3458,2832,3846,2462,4427,4389,3161,2770,...,2.55000,2.6250,2.3775,3.362500,1.897500,2.772500,2.420000,2.37625,2.704286,200


In [3]:
df[df.columns[df.isna().any()].tolist()].isna().sum()

championLevel100_1      3
championLevel100_3      1
championLevel100_4      2
championLevel100_5      1
championLevel200_6      1
championLevel200_8      2
championLevel200_10     2
championPoints100_1     3
championPoints100_3     1
championPoints100_4     2
championPoints100_5     1
championPoints200_6     1
championPoints200_8     2
championPoints200_10    2
dtype: int64

In [4]:
df[df['championLevel100_5'].isna()]['matchId']


5413    5870
Name: matchId, dtype: int64

In [5]:
#2. try by filling missing values with 0's--works better
df2 = df.fillna(0)

In [6]:
#3. try with missing values dropped
df3 = df.dropna()

In [7]:
df3.shape

(7988, 202)

In [8]:
#trying with nan columns dropped
df4 = df.drop(['championLevel100_4',
 'championLevel200_7',
 'championPoints100_4',
 'championPoints200_7', 'championLevel100_1',
 'championLevel100_3',

 'championLevel100_5',
 'championLevel200_6',
 'championLevel200_8',
 'championLevel200_10',
 'championPoints100_1',
 'championPoints100_3',
 
 'championPoints100_5',
 'championPoints200_6',
 'championPoints200_8',
 'championPoints200_10'], axis = 1)

from https://www.analyticsvidhya.com/blog/2021/09/gradient-boosting-algorithm-a-complete-guide-for-beginners/

In [9]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
import warnings
warnings.filterwarnings("ignore")

In [10]:
#categorical columns need special treatment:
categoricals = [col for col in df2 if col.startswith('championName')]
categoricals

['championName100_1',
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10']

In [11]:
#X[categoricals] = X[categoricals].astype('category')

In [12]:
X = df2.drop(['winner',  'championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [13]:
y = df2['winner']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=3)

In [15]:
X_train.shape

(6400, 191)

In [16]:
gbc=GradientBoostingClassifier(n_estimators=500,learning_rate=0.05,random_state=100,max_features=5 )

In [17]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [18]:
print(confusion_matrix(y_test, gbc.predict(X_test)))

[[622 197]
 [241 540]]


In [19]:
print("GBC accuracy is %2.4f" % accuracy_score( 
     y_test, gbc.predict(X_test)))

GBC accuracy is 0.7262


'gbc' gives 72.62
-compared to last nb v14_.. 72.4

In [23]:
#Loading Hayden's data
underTenTest = pd.read_csv('../data/underTenTest.csv')
underTen = pd.read_csv('../data/underTenTrain.csv')

In [24]:
df5 = df2.drop(['championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [74]:
X = df5.drop(['winner'], axis  =1)
X=X.loc[~X['matchId'].isin(underTen['matchId'])]

In [75]:
y = df5.loc[~df5['matchId'].isin(underTen['matchId'])]['winner']

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [77]:
gbc=GradientBoostingClassifier(n_estimators=500,learning_rate=0.05,random_state=100,max_features=5 )

In [78]:
gbc.fit(X_train,y_train)

GradientBoostingClassifier(learning_rate=0.05, max_features=5, n_estimators=500,
                           random_state=100)

In [79]:
print("GBC accuracy is %2.4f" % accuracy_score( 
     y_test, gbc.predict(X_test)))

GBC accuracy is 0.7112


In [33]:
print(confusion_matrix(y_test, gbc.predict(X_test)))

[[763   0]
 [  0 802]]


In [34]:
from sklearn.metrics import classification_report

pred=gbc.predict(X_test)

print(classification_report(y_test, pred))


              precision    recall  f1-score   support

         100       1.00      1.00      1.00       763
         200       1.00      1.00      1.00       802

    accuracy                           1.00      1565
   macro avg       1.00      1.00      1.00      1565
weighted avg       1.00      1.00      1.00      1565



**Tuning

In [21]:
from sklearn.model_selection import GridSearchCV

grid = {

    'learning_rate':[0.01,0.05,0.1],

    'n_estimators':np.arange(100,500,100),

}


gb = GradientBoostingClassifier()

gb_cv = GridSearchCV(gb, grid, cv = 4)

gb_cv.fit(X_train,y_train)

print("Best Parameters:",gb_cv.best_params_)

print("Train Score:",gb_cv.best_score_)

print("Test Score:",gb_cv.score(X_test,y_test))

Best Parameters: {'learning_rate': 0.05, 'n_estimators': 400}
Train Score: 0.7215625
Test Score: 0.725625


gb_cv gives 72.15

**Tuning max_depth**

gb_cv_md - use gb_cv (72.15) and tune for max depth 

In [31]:
grid = {'max_depth':[2,3,4,5,6,7] }

gb = GradientBoostingClassifier(learning_rate=0.1,n_estimators=100)

gb_cv_md = GridSearchCV(gb, grid, cv = 4)

gb_cv_md.fit(X_train,y_train)

print("Best Parameters:",gb_cv_md.best_params_)

print("Train Score:",gb_cv_md.best_score_)

print("Test Score:",gb_cv_md.score(X_test,y_test))

Best Parameters: {'max_depth': 3}
Train Score: 0.7183110071057827
Test Score: 0.72090112640801


gbc_cv_md --tuned for max depth gives 72.09
**--using gb_cv (tuned for learning rate, n_estimators)**

# Predict on Test data

In [35]:
folder_path = '../data/test_timelines/'
file_list = glob.glob(os.path.join(folder_path, '*.json'))

In [36]:
participantid = []
currentgold = []
totalgold = []
minionskilled = []
xp = []
matchid = []
teamid = []
cs_abilityHaste = []
cs_abilityPower = []
cs_armor = []
cs_attackDamage = []
cs_attackSpeed = []
cs_health = []
cs_healthMax = []
dd_totalDamageTaken = []
dd_pDamageDoneToChamps = []
dd_totDamageDone = []
positionX = []
positionY = []



def file_buster(file_path):
    with open(file_path) as json_file:
        data = json.load(json_file)
    matchida= data['matchId']
    for i, j in data['frames'][-1]['participantFrames'].items():  #looks at last frame, looks at participantFrames equiv to last_stats
        participantid.append(i)
        currentgold.append(j['currentGold'])
        totalgold.append(j['totalGold'])
        xp.append(j['xp'])
        minionskilled.append(j['minionsKilled'])
        matchid.append(matchida)
        if int(i) < 6:
            teamid.append('100')
        else:
            teamid.append('200')
        cs_stats = j['championStats']
        cs_abilityHaste.append(cs_stats['abilityHaste'])
        cs_abilityPower.append(cs_stats['abilityPower'])
        cs_armor.append(cs_stats['armor'])
        cs_attackDamage.append(cs_stats['attackDamage'])
        cs_attackSpeed.append(cs_stats['attackSpeed'])
        cs_health.append(cs_stats['health'])
        cs_healthMax.append(cs_stats['healthMax'])
    
        d_stats = j['damageStats']
        dd_totalDamageTaken.append(d_stats['totalDamageTaken'])
        dd_pDamageDoneToChamps.append(d_stats['totalDamageDoneToChampions'])
        dd_totDamageDone.append(d_stats['totalDamageDone'])
    
        pos = j['position']
        positionX.append(pos['x'])
        positionY.append(pos['y']) 


In [37]:
count = 0
for file in file_list:
    #print('loading file: ' + str(count))
    file_buster(file)
    count+=1
print(str(count) + " files loaded")

2000 files loaded


In [38]:
di = {'matchId':matchid, 'teamId':teamid, 'participantId' : participantid, 'currentGold': currentgold, \
      'totalGold':totalgold, 'xp': xp, 'cs_abilityHaste': cs_abilityHaste, 'cs_abilityPower':cs_abilityPower,\
     'cs_armor': cs_armor, 'cs_attackDamage': cs_attackDamage,'cs_attackSpeed':cs_attackSpeed, \
     'cs_health': cs_health, 'cs_healthMax': cs_healthMax, 'dd_totalDamageTaken':dd_totalDamageTaken, \
     'dd_pDamageDoneToChamps' : dd_pDamageDoneToChamps, 'dd_totDamageDone': dd_totDamageDone, \
     'positionX': positionX, 'positionY': positionY}
za = pd.DataFrame(di)
za    

,matchId,teamId,participantId,currentGold,totalGold,xp,cs_abilityHaste,cs_abilityPower,cs_armor,cs_attackDamage,cs_attackSpeed,cs_health,cs_healthMax,dd_totalDamageTaken,dd_pDamageDoneToChamps,dd_totDamageDone,positionX,positionY
0,8000,100,1,1513,3063,4659,0,0,66,110,128,774,1342,3513,1159,21426,3127,12922
1,8000,100,2,1742,4117,3539,0,0,65,141,138,535,1153,8469,3660,45005,7949,8549
2,8000,100,3,1015,4365,4431,0,191,57,67,112,1451,1518,1949,2166,29146,8052,8160
3,8000,100,4,80,3555,3061,0,0,49,131,183,1077,1077,2352,3032,17927,11236,1751
4,8000,100,5,78,2213,2409,0,46,51,61,108,1081,1214,2633,1062,2604,11482,1563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,217,2852,4621,0,69,67,84,115,1052,1475,5129,3780,19606,3633,12989
19996,9999,200,7,1510,2860,2905,0,58,58,65,108,885,1068,8419,1111,48297,7778,10713
19997,9999,200,8,495,3245,4214,0,0,50,113,245,295,1339,5560,3940,22466,6292,7209
19998,9999,200,9,283,2483,2565,0,0,44,108,113,952,1106,1824,903,15240,13859,4119


In [39]:
champion_mastery = pd.read_csv('../data/champion_mastery.csv')
participants_test = pd.read_csv('../data/participants_test.csv')

In [40]:
champion_points = pd.merge(participants_test, champion_mastery, on = ['summonerId', 'championId'], how = 'left' ).drop(['chestGranted', 'tokensEarned'], axis = 1)
champion_points

,matchId,teamId,participantId,summonerId,summonerLevel,championName,championId,championLevel,championPoints
0,8000,100,1,23938,57,Fiora,114,3.0,10896.0
1,8000,100,2,8037,788,LeeSin,64,7.0,143994.0
2,8000,100,3,8444,566,Veigar,45,7.0,203012.0
3,8000,100,4,24831,542,Twitch,29,7.0,376388.0
4,8000,100,5,23890,224,Soraka,16,7.0,625009.0
...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,45320,458,Mordekaiser,82,7.0,117779.0
19996,9999,200,7,45321,330,FiddleSticks,9,5.0,22922.0
19997,9999,200,8,45322,462,Tristana,18,7.0,169934.0
19998,9999,200,9,45323,256,Samira,360,4.0,19716.0


In [41]:
#convert to int types
coltomergeon = ['matchId', 'teamId', 'participantId']

for x in za[coltomergeon]:
    za[x] = za[x].astype(int)
    
for x in champion_points[coltomergeon]:
    champion_points[x] = champion_points[x].astype(int)

In [42]:
test_data = pd.merge(za, champion_points, on = ['matchId', 'teamId', 'participantId'], how = 'left')
test_data

,matchId,teamId,participantId,currentGold,totalGold,xp,cs_abilityHaste,cs_abilityPower,cs_armor,cs_attackDamage,...,dd_pDamageDoneToChamps,dd_totDamageDone,positionX,positionY,summonerId,summonerLevel,championName,championId,championLevel,championPoints
0,8000,100,1,1513,3063,4659,0,0,66,110,...,1159,21426,3127,12922,23938,57,Fiora,114,3.0,10896.0
1,8000,100,2,1742,4117,3539,0,0,65,141,...,3660,45005,7949,8549,8037,788,LeeSin,64,7.0,143994.0
2,8000,100,3,1015,4365,4431,0,191,57,67,...,2166,29146,8052,8160,8444,566,Veigar,45,7.0,203012.0
3,8000,100,4,80,3555,3061,0,0,49,131,...,3032,17927,11236,1751,24831,542,Twitch,29,7.0,376388.0
4,8000,100,5,78,2213,2409,0,46,51,61,...,1062,2604,11482,1563,23890,224,Soraka,16,7.0,625009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,9999,200,6,217,2852,4621,0,69,67,84,...,3780,19606,3633,12989,45320,458,Mordekaiser,82,7.0,117779.0
19996,9999,200,7,1510,2860,2905,0,58,58,65,...,1111,48297,7778,10713,45321,330,FiddleSticks,9,5.0,22922.0
19997,9999,200,8,495,3245,4214,0,0,50,113,...,3940,22466,6292,7209,45322,462,Tristana,18,7.0,169934.0
19998,9999,200,9,283,2483,2565,0,0,44,108,...,903,15240,13859,4119,45323,256,Samira,360,4.0,19716.0


In [55]:
metasrc = pd.read_csv('../data/metasrc.csv')
metasrc['winrt'] = metasrc['Win %'].str.rstrip('%').astype('float')
cols_to_ag = ['Score', 'winrt', 'KDA']

meta_aggs = pd.DataFrame({'Name': metasrc['Name'].unique()})


for col in metasrc[cols_to_ag]:
    avg = pd.DataFrame(metasrc.groupby(['Name'])[col].mean())
    meta_aggs = pd.merge(meta_aggs, avg, how = 'inner', on = 'Name')
    
meta_aggs
meta_aggs['Name'] = meta_aggs['Name'].str.replace(r'(\w+)\1', r'\1')
meta_aggs = meta_aggs.rename(columns = {'Name' : 'championName'})
meta_aggs = meta_aggs.replace({'championName':{'Le SinLe Sin':'LeeSin', 'Master YiMaster Yi' : 'MasterYi',"Kai'SaKai'Sa": 'Kaisa', \
                                    'Aurelion SolAurelion Sol': 'AurelionSol', 'LeBlanc':'Leblanc', 
                                    'Renata GlascRenata Glasc':'Renata',
                                    "Rek'SaiRek'Sai": 'RekSai',
                                    'Twisted FateTwisted Fate':'TwistedFate',
                                    'Mis FortuneMis Fortune':'MissFortune',
                                    "Kha'ZixKha'Zix":'Khazix',
                                    "Cho'GathCho'Gath":'Chogath',
                                    'Jarvan IVJarvan IV':'JarvanIV',
                                    'Xin ZhaoXin Zhao':'XinZhao',
                                    "Bel'VethBel'Veth":'Belveth',
                                    "Kog'MawKog'Maw": 'KogMaw',
                                    'Fiddlesticks':'FiddleSticks',
                                    'Tahm KenchTahm Kench':'TahmKench',
                                    "Vel'KozVel'Koz":'Velkoz',
                                    'Dr. MundoDr. Mundo': 'DrMundo',
                                    "K'SanteK'Sante":'KSante', 'Wukong': 'MonkeyKing'
                                               
                                   }})

In [56]:
test_data = pd.merge(test_data, meta_aggs, on = 'championName', how = 'left')

In [57]:
results = pd.DataFrame({'matchId': test_data['matchId'].unique()})

cols_to_pivot = ['totalGold', 'xp',  'cs_abilityHaste', 'cs_abilityPower','cs_armor', 'cs_attackDamage','cs_attackSpeed', \
     'cs_health', 'cs_healthMax', 'dd_totalDamageTaken', \
     'dd_pDamageDoneToChamps' ,'dd_totDamageDone', \
     'positionX', 'positionY' , 'championName', 'championLevel', 'championPoints',
                'Score', 'winrt', 'KDA']


for i in test_data[cols_to_pivot]:
    print(i)
    pivot_table = pd.pivot(test_data, values = i, index = 'matchId', columns = ['teamId', 'participantId'])
    
    pivot_table.columns = [str(i)+str('_'.join(map(str, c))) for c in pivot_table]
    
    results = pd.merge(results, pivot_table, on='matchId', how = 'inner')
    print(results.shape)
    
   

totalGold
(2000, 11)
xp
(2000, 21)
cs_abilityHaste
(2000, 31)
cs_abilityPower
(2000, 41)
cs_armor
(2000, 51)
cs_attackDamage
(2000, 61)
cs_attackSpeed
(2000, 71)
cs_health
(2000, 81)
cs_healthMax
(2000, 91)
dd_totalDamageTaken
(2000, 101)
dd_pDamageDoneToChamps
(2000, 111)
dd_totDamageDone
(2000, 121)
positionX
(2000, 131)
positionY
(2000, 141)
championName
(2000, 151)
championLevel
(2000, 161)
championPoints
(2000, 171)
Score
(2000, 181)
winrt
(2000, 191)
KDA
(2000, 201)


In [58]:
results.shape

(2000, 201)

In [59]:
results

,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,KDA100_1,KDA100_2,KDA100_3,KDA100_4,KDA100_5,KDA200_6,KDA200_7,KDA200_8,KDA200_9,KDA200_10
0,8000,3063,4117,4365,3555,2213,3075,3884,3024,4150,...,1.73500,2.945000,2.625000,2.31125,2.9900,1.820833,2.809167,2.131250,2.37750,2.057500
1,8001,3784,3548,3203,4193,2764,3048,4045,2582,4168,...,1.93375,2.945000,2.277500,2.51250,3.2600,1.952500,2.466250,3.031250,2.32000,2.704286
2,8002,2681,4605,4309,2569,2042,3094,2827,3971,3504,...,2.59375,2.620000,2.809167,2.66750,2.7100,2.168333,2.697500,2.802500,2.40000,3.131667
3,8003,3471,3808,4186,4028,2291,4803,4072,2681,3130,...,2.13125,2.975000,2.257692,3.17750,2.4450,1.915000,2.452857,2.606250,2.40000,2.704286
4,8004,3181,3569,3693,3274,2624,3210,3997,3127,4044,...,1.84750,2.162500,2.442500,2.65500,3.2600,2.602500,1.390000,3.015000,2.51250,2.445000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,3735,2545,4207,2930,2023,2992,3100,3282,3288,...,2.01250,3.146667,2.885000,2.55000,2.9900,1.882500,2.628182,2.700000,2.31125,3.056000
1996,9996,4162,4277,3313,4289,3082,3253,3173,4187,4079,...,1.78600,2.945000,1.820833,2.38000,2.3200,1.952500,3.090000,2.447500,2.37625,3.260000
1997,9997,2747,3392,2755,3269,2556,3728,3391,3492,4135,...,1.74250,2.745000,2.447500,2.88500,3.0560,1.725000,2.292500,2.277500,2.51250,4.212500
1998,9998,3538,3566,3653,4623,2973,2846,3629,2856,2931,...,1.91500,2.768000,2.447500,1.99000,3.0150,1.725000,2.820000,1.820833,2.51250,2.704286


In [60]:
patchByLaneTest = pd.read_csv('../data/patchByLaneTest.csv')
patchByLaneTest

,matchId,champWinPercent100_1,champWinPercent100_2,champWinPercent100_3,champWinPercent100_4,champWinPercent100_5,champWinPercent200_6,champWinPercent200_7,champWinPercent200_8,champWinPercent200_9,...,champScore100_1,champScore100_2,champScore100_3,champScore100_4,champScore100_5,champScore200_6,champScore200_7,champScore200_8,champScore200_9,champScore200_10
0,8000,0.5250,0.5138,0.5285,0.5168,0.5189,0.4919,0.5237,0.5008,0.4970,...,70.16,94.80,58.69,44.49,54.89,62.13,64.41,68.07,84.23,44.20
1,8001,0.4994,0.5138,0.4559,0.5333,0.5144,0.5173,0.4978,0.4949,0.5237,...,45.07,94.80,41.48,79.18,68.98,76.08,50.09,46.60,47.14,56.38
2,8002,0.4961,0.4837,0.5234,0.5222,0.5131,0.5193,0.5132,0.5145,0.4900,...,42.67,61.51,48.59,68.26,54.52,62.28,48.00,55.34,54.69,46.80
3,8003,0.5008,0.5013,0.5482,0.5221,0.5354,0.4912,0.4940,0.5387,0.4900,...,68.07,49.74,50.46,63.75,56.54,55.32,49.11,44.79,54.69,56.38
4,8004,0.5202,0.4753,0.4903,0.5295,0.5144,0.4883,0.5047,0.4956,0.5333,...,50.70,41.35,47.55,48.84,68.98,46.56,47.06,55.35,79.18,56.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,0.5070,0.4952,0.5072,0.4884,0.5136,0.4849,0.5190,0.5113,0.5241,...,60.06,55.79,58.89,66.87,62.45,49.47,53.91,50.11,47.33,66.08
1996,9996,0.4894,0.5138,0.4919,0.5210,0.4346,0.5173,0.5100,0.5035,0.5215,...,55.02,94.80,62.13,37.49,35.06,76.08,51.70,65.25,45.29,68.98
1997,9997,0.4801,0.5057,0.5035,0.5120,0.4915,0.5160,0.5080,0.4559,0.5333,...,59.11,49.29,65.25,58.64,54.17,90.92,58.21,41.48,79.18,96.39
1998,9998,0.4912,0.4795,0.5035,0.5167,0.5000,0.5160,0.4893,0.4919,0.5333,...,55.32,38.89,65.25,43.11,45.59,90.92,53.95,62.13,79.18,56.38


In [61]:
#adding patch test
results_p = pd.merge(results, patchByLaneTest, how = 'inner', on = 'matchId')
results_p

,matchId,totalGold100_1,totalGold100_2,totalGold100_3,totalGold100_4,totalGold100_5,totalGold200_6,totalGold200_7,totalGold200_8,totalGold200_9,...,champScore100_1,champScore100_2,champScore100_3,champScore100_4,champScore100_5,champScore200_6,champScore200_7,champScore200_8,champScore200_9,champScore200_10
0,8000,3063,4117,4365,3555,2213,3075,3884,3024,4150,...,70.16,94.80,58.69,44.49,54.89,62.13,64.41,68.07,84.23,44.20
1,8001,3784,3548,3203,4193,2764,3048,4045,2582,4168,...,45.07,94.80,41.48,79.18,68.98,76.08,50.09,46.60,47.14,56.38
2,8002,2681,4605,4309,2569,2042,3094,2827,3971,3504,...,42.67,61.51,48.59,68.26,54.52,62.28,48.00,55.34,54.69,46.80
3,8003,3471,3808,4186,4028,2291,4803,4072,2681,3130,...,68.07,49.74,50.46,63.75,56.54,55.32,49.11,44.79,54.69,56.38
4,8004,3181,3569,3693,3274,2624,3210,3997,3127,4044,...,50.70,41.35,47.55,48.84,68.98,46.56,47.06,55.35,79.18,56.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,9995,3735,2545,4207,2930,2023,2992,3100,3282,3288,...,60.06,55.79,58.89,66.87,62.45,49.47,53.91,50.11,47.33,66.08
1996,9996,4162,4277,3313,4289,3082,3253,3173,4187,4079,...,55.02,94.80,62.13,37.49,35.06,76.08,51.70,65.25,45.29,68.98
1997,9997,2747,3392,2755,3269,2556,3728,3391,3492,4135,...,59.11,49.29,65.25,58.64,54.17,90.92,58.21,41.48,79.18,96.39
1998,9998,3538,3566,3653,4623,2973,2846,3629,2856,2931,...,55.32,38.89,65.25,43.11,45.59,90.92,53.95,62.13,79.18,56.38


In [62]:
dftest = results_p.dropna()

In [49]:
dftest.shape

(1998, 191)

In [63]:
testing_x = dftest.drop(['championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [64]:
testing_nas = results_p.drop(['championName100_1',  #trying it without categoricals
 'championName100_2',
 'championName100_3',
 'championName100_4',
 'championName100_5',
 'championName200_6',
 'championName200_7',
 'championName200_8',
 'championName200_9',
 'championName200_10'], axis = 1)

In [65]:
testing_nas.shape

(2000, 211)

In [66]:
testing_nas.columns[testing_nas.isna().any()].tolist()

['championLevel100_4',
 'championLevel200_7',
 'championPoints100_4',
 'championPoints200_7']

In [67]:

testing_nas = testing_nas.fillna(0)  

In [68]:
testing_nas =testing_nas.loc[~testing_nas['matchId'].isin(underTenTest['matchId'])]

In [71]:
testing_nas = testing_nas.drop(['champScore100_1',
'champScore100_2'
,'champScore100_3'
,'champScore100_4'
,'champScore100_5'], axis =1)

In [72]:
subm_y = gbc.predict(testing_nas)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- champScore200_10
- champScore200_6
- champScore200_7
- champScore200_8
- champScore200_9
- ...
Feature names seen at fit time, yet now missing:
- winner


In [49]:
subm_y = gb_cv.predict(testing_nas)

In [50]:
subm_y

array([100, 200, 100, ..., 200, 100, 100], dtype=int64)

In [51]:
len(subm_y)

2000

In [52]:
submission = pd.read_csv('../data/sample_submission.csv')
submission

,matchId,winner
0,8000,100
1,8001,100
2,8002,100
3,8003,100
4,8004,100
...,...,...
1995,9995,100
1996,9996,100
1997,9997,100
1998,9998,100


In [53]:
submission['winner'] = subm_y
submission

,matchId,winner
0,8000,100
1,8001,200
2,8002,100
3,8003,100
4,8004,200
...,...,...
1995,9995,100
1996,9996,100
1997,9997,200
1998,9998,100


In [54]:
submission.to_csv('../data/submissionV14_3_31.csv', index = False)